In [1]:
#######
# Copyright 2020 Jian Zhang, All rights reserved
##
!pip install pretty_midi
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

from math import ceil
velo_inc = 5
dim = 128*2 + 100 + int(ceil(126/velo_inc))  # This is the size of vocabulary.

import random
import glob
import numpy as np
import pretty_midi

class Event:
    def __init__(self, s, t, v):
        self.time = s
        self.type = t
        self.val = v

    def encode(self):
        if self.type == 'down':
            return self.val
        elif self.type == 'up':
            return 128 + self.val
        elif self.type == 'shift':
            return 128*2 + self.val
        else:
            return 128*2 + 100 + self.val

    @staticmethod
    def decode(code):
        if code < 128:
            return 'down', code
        elif code < 128*2:
            return 'up', code - 128
        elif code < 128*2 + 100:
            return 'shift', (code - 128*2)/100 + 0.01
        else:
            return 'velo', (code - 128*2 - 100)*velo_inc + int(velo_inc/2)


def piano2seq(midi):
    '''
    Convert a midi object to a sequence of events
    :param midi: midi object or the file name of the midi file
    :return: numpy array that contains the sequence of events
    '''
    if type(midi) is str:
        midi = pretty_midi.PrettyMIDI(midi)
    piano = midi.instruments[0]

    velo = 0
    q = []
    for note in piano.notes:
        if note.velocity != velo:
            q.append(Event(note.start, 'velo', int(min(note.velocity, 125)/velo_inc)))
            velo = note.velocity
        q.append(Event(note.start, 'down', note.pitch))
        q.append(Event(note.end, 'up', note.pitch))

    t = 0
    qfull = []
    for e in sorted(q, key=lambda x: x.time):
        d = e.time - t
        while d > 0.01:
            dd = min(d, 1) - 0.01
            qfull.append(Event(t, 'shift', int(dd*100)))
            d = d - dd
        t = e.time
        qfull.append(e)

    seq = np.zeros((len(qfull),), dtype=np.int32)
    for i, e in enumerate(qfull):
        seq[i] = e.encode()

    assert np.max(seq) < dim
    return seq

def seq2piano(seq):
    '''
    Convert a sequence of events to midi
    :param seq: numpy array that contains the sequence
    :return: midi object
    '''
    midi = pretty_midi.PrettyMIDI()
    piano = pretty_midi.Instrument(program=0, is_drum=False, name='piano')
    midi.instruments.append(piano)

    if seq.ndim > 1:
        seq = np.argmax(seq, axis=-1)
    inote = {}
    velo = 40
    time = 0.
    for e in seq:
        t, v = Event.decode(e)
        if t == 'shift':
            time += v
        elif t == 'velo':
            velo = v
            for n in inote.values():
                if n[2] == time:
                    n[0] = v
        elif t == 'down':
            n = inote.get(v, None)
            if n is not None:
                logging.debug('consecutive downs for pitch %d at time %d and %d' % (v, n[2], time))
            else:
                inote[v]  = [velo, v, time, -1]
        else:
            n = inote.get(v, None)
            if n is not None:
                n[-1] = time
                if n[-1] > n[-2]:
                    piano.notes.append(pretty_midi.Note(*n))
                else:
                    logging.debug('note with non-positive duration for pitch %d at time %d' % (n[1], n[2]))
                del inote[v]
            else:
                logging.debug('up without down for pitch %d at time %d' % (v, time))

    # clean out the incomplete note buffer, assuming these note end at last
    for n in inote.values():
        n[-1] = time
        if n[-1] > n[-2]:
            piano.notes.append(pretty_midi.Note(*n))

    return midi

def segment(seq, maxlen=150):
    assert len(seq) > maxlen
    inc = int(maxlen/2)
    i = inc
    t = np.ones((maxlen+1,), dtype=np.int32)
    t[0] = (128*2+1)
    t[1:] = seq[:maxlen]
    s = [t]
    while i+maxlen+1 < len(seq):
        s.append(seq[i:i+maxlen+1])
        i += inc
    return np.stack(s, axis=0)

def process_midi_seq(all_midis=None, datadir='data', n=10000, maxlen=150):
    '''
    Process a list of midis, convert them to sequences and segment sequences into segments of length max_len
    :param all_midis: the list of midis. If None, midis will be loaded from files
    :param datadir: data directory, assume under this directory, we have the "maestro-v1.0.0" midi directory
    :param n: # of segments to return
    :param maxlen: the length of the segments
    :return: numpy array of shape [n', max_len] for the segments. n' tries to be close to n but may not be exactly n.
    '''
    if all_midis is None:
        all_midis = glob.glob(datadir+'/maestro-v1.0.0/**/*.midi')
        random.seed()    # for debug purpose, you can pass a fix number when calling seed()
        random.shuffle(all_midis)

    data = []
    k = 0
    for m in all_midis:
        seq = segment(piano2seq(m), maxlen)
        data.append(seq)
        k += len(seq)
        if k > n:
            break

    return np.vstack(data)

def random_piano(n=100):
    '''
    Generate random piano note
    :param n: # of notes to be generated
    :return: midi object with the notes
    '''
    midi = pretty_midi.PrettyMIDI()
    piano = pretty_midi.Instrument(program=0, is_drum=False, name='piano')
    midi.instruments.append(piano)

    pitchs = np.random.choice(128, size=n)
    velos = np.random.choice(np.arange(10, 80), size=n)
    durations = np.abs(np.random.randn(n) + 1)
    intervs = np.abs(0.2*np.random.randn(n) + 0.3)
    time = 0.5
    for i in range(n):
        piano.notes.append(pretty_midi.Note(velos[i], pitchs[i], time, time+durations[i]))
        time += intervs[i]

    return midi


In [2]:
#######
# Copyright 2020 Jian Zhang, All rights reserved
##
import logging
from abc import ABC, abstractmethod

class ModelBase(ABC):
    @abstractmethod
    def __init__(self, load_trained=False):
        '''
        :param load_trained
            If load_trained is True, load a trained model from a file.
            Should include code to download the file from Google drive if necessary.
            else, construct the model
        '''
        if load_trained:
            logging.info('load model from file ...')

    @abstractmethod
    def train(self, x):
        '''
        Train the model on one batch of data
        :param x: train data. For composer training, a single torch tensor will be given
        and for critic training, x will be a tuple of two tensors (data, label)
        :return: (mean) loss of the model on the batch
        '''
        pass

class ComposerBase(ModelBase):
    '''
    Class wrapper for a model that can be trained to generate music sequences.
    '''
    @abstractmethod
    def compose(self, n):
        '''
        Generate a music sequence
        :param n: length of the sequence to be generated
        :return: the generated sequence
        '''
        pass

class CriticBase(ModelBase):
    '''
    Class wrapper for a model that can be trained to criticize music sequences.
    '''
    @abstractmethod
    def score(self, x):
        '''
        Compute the score of a music sequence
        :param x: a music sequence
        :return: the score between 0 and 1 that reflects the quality of the music: the closer to 1, the better
        '''
        pass

In [3]:
#######
# Copyright 2020 Jian Zhang, All rights reserved
##
import logging
from abc import ABC, abstractmethod

class ModelBase(ABC):
    @abstractmethod
    def __init__(self, load_trained=False):
        '''
        :param load_trained
            If load_trained is True, load a trained model from a file.
            Should include code to download the file from Google drive if necessary.
            else, construct the model
        '''
        if load_trained:
            logging.info('load model from file ...')

    @abstractmethod
    def train(self, x):
        '''
        Train the model on one batch of data
        :param x: train data. For composer training, a single torch tensor will be given
        and for critic training, x will be a tuple of two tensors (data, label)
        :return: (mean) loss of the model on the batch
        '''
        pass

class ComposerBase(ModelBase):
    '''
    Class wrapper for a model that can be trained to generate music sequences.
    '''
    @abstractmethod
    def compose(self, n):
        '''
        Generate a music sequence
        :param n: length of the sequence to be generated
        :return: the generated sequence
        '''
        pass

class CriticBase(ModelBase):
    '''
    Class wrapper for a model that can be trained to criticize music sequences.
    '''
    def __init__(self):
        super(CriticBase, self).__init__()

    @abstractmethod
    def score(self, x):
        '''
        Compute the score of a music sequence
        :param x: a music sequence
        :return: the score between 0 and 1 that reflects the quality of the music: the closer to 1, the better
        '''
        pass


In [4]:
import requests
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pretty_midi
import glob
import requests
import zipfile
import os

data_url = "https://storage.googleapis.com/magentadata/datasets/maestro/v1.0.0/maestro-v1.0.0-midi.zip"
data_path = "maestro-v1.0.0-midi.zip"


response = requests.get(data_url, stream=True)
with open(data_path, 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

with zipfile.ZipFile(data_path, 'r') as zip_ref:
    zip_ref.extractall()

os.remove(data_path)
good_sequences = glob.glob('maestro-v1.0.0/**/*.midi', recursive=True)

In [5]:
len(good_sequences)

1184

In [6]:
good_music = process_midi_seq(good_sequences)

# Generate bad music and convert them to sequences
bad_sequences = []
midi = random_piano(40000)  # Assuming the function can be modified to generate sequences of desired length
bad_sequences = process_midi_seq([midi])  # This function should convert midi to a sequence of events of shape (51,)



In [7]:
good_music.shape,bad_sequences.shape

((10008, 151), (3309, 151))

In [20]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

class Critic(nn.Module):
    def __init__(self, class_weights=None):
        super(Critic, self).__init__()

        # Multi-layer LSTM with dropout for regularization
        self.lstm = nn.LSTM(input_size=151, hidden_size=100, num_layers=3,
                    batch_first=True, dropout=0.5)
        self.fc = nn.Linear(100, 1)  # Adjust `num_classes` as necessary


        # BCEWithLogitsLoss already includes sigmoid activation
        self.criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights)
        self.optimizer = optim.Adam(self.parameters(), lr=0.0001)

    def forward(self, x):
        out, _ = self.lstm(x)
        if len(out.shape) == 3:
            out = out[:, -1, :]
        out = self.fc(out)
        return out  # Removed sigmoid here as BCEWithLogitsLoss will apply it

    def train_model1(self, x, label):
        self.optimizer.zero_grad()
        outputs = self.forward(x)
        # Apply sigmoid before calculating loss
        loss = self.criterion(outputs, label)
        loss.backward()
        self.optimizer.step()
        return loss.item()

    def score(self, x):
        with torch.no_grad():
            # Apply sigmoid to map output scores to [0, 1] range
            return torch.sigmoid(self.forward(x)).cpu().numpy()  # Return as NumPy array for batch processing



In [9]:
good_music.shape,bad_sequences.shape

((10008, 151), (3309, 151))

In [10]:
type(good_music),type(bad_sequences)

(numpy.ndarray, numpy.ndarray)

In [11]:
# 标签准备
bad_labels = torch.zeros(len(bad_sequences), 1)
good_labels = torch.ones(len(good_music), 1)
bad_music_tensor = torch.tensor(bad_sequences)
good_music_tensor = torch.tensor(good_music)

# 数据和标签的合并
all_data = torch.cat([bad_music_tensor, good_music_tensor], 0)
all_labels = torch.cat([bad_labels, good_labels], dim=0)

# 数据加载和批处理
dataset = TensorDataset(all_data, all_labels)

from torch.utils.data import random_split

train_size = int(0.7 * len(dataset))  # 70% of the dataset for training
remaining_size = len(dataset) - train_size
valid_size = int(0.5 * remaining_size)  # Split the remaining 30% equally for validation and test
test_size = remaining_size - valid_size

# Split the dataset
train_dataset, remaining_dataset = random_split(dataset, [train_size, remaining_size])
valid_dataset, test_dataset = random_split(remaining_dataset, [valid_size, test_size])

# Create dataloaders for each set
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [12]:
all_data.shape

torch.Size([13317, 151])

In [21]:
import torch.optim as optim


# Initialize the model
model1 = Critic()
num_epochs = 100

for epoch in range(num_epochs):
    model1.train()  # Set the model to training mode
    total_train_loss = 0.0

    # Training
    for batch_data, batch_labels in train_loader:
        batch_data = batch_data.float()
        batch_labels = batch_labels.float()

        loss = model1.train_model1(batch_data, batch_labels)
        total_train_loss += loss

    average_train_loss = total_train_loss / len(train_loader)

    # Validation
    model1.eval()  # Set the model to evaluation mode
    total_val_loss = 0.0
    total_val_correct = 0
    with torch.no_grad():
        for batch_data, batch_labels in valid_loader:
            batch_data = batch_data.float()
            batch_labels = batch_labels.float()

            outputs = model1(batch_data)
            val_loss = model1.criterion(outputs, batch_labels)
            total_val_loss += val_loss.item()

            # Calculate accuracy
            predicted = (outputs > 0.5).float()
            total_val_correct += (predicted == batch_labels).sum().item()

    average_val_loss = total_val_loss / len(valid_loader)
    val_accuracy = total_val_correct / (len(valid_loader.dataset))
    print(f"Epoch {epoch+1}, Training Loss: {average_train_loss:.4f}, Validation Loss: {average_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")


Epoch 1, Training Loss: 0.5980, Validation Loss: 0.5602, Validation Accuracy: 0.7357
Epoch 2, Training Loss: 0.5665, Validation Loss: 0.5621, Validation Accuracy: 0.7417
Epoch 3, Training Loss: 0.5682, Validation Loss: 0.5590, Validation Accuracy: 0.7417
Epoch 4, Training Loss: 0.5668, Validation Loss: 0.5586, Validation Accuracy: 0.7417
Epoch 5, Training Loss: 0.5667, Validation Loss: 0.5580, Validation Accuracy: 0.7457
Epoch 6, Training Loss: 0.5646, Validation Loss: 0.5582, Validation Accuracy: 0.7487
Epoch 7, Training Loss: 0.5661, Validation Loss: 0.5579, Validation Accuracy: 0.7487
Epoch 8, Training Loss: 0.5647, Validation Loss: 0.5572, Validation Accuracy: 0.7487
Epoch 9, Training Loss: 0.5629, Validation Loss: 0.5581, Validation Accuracy: 0.7487
Epoch 10, Training Loss: 0.5652, Validation Loss: 0.5571, Validation Accuracy: 0.7487
Epoch 11, Training Loss: 0.5640, Validation Loss: 0.5573, Validation Accuracy: 0.7487
Epoch 12, Training Loss: 0.5632, Validation Loss: 0.5569, Valid

In [22]:
correct = 0
total = 0

# Iterate over batches in the test_loader
for batch_data, batch_labels in test_loader:
    batch_data = batch_data.float()
    scores = model1.forward(batch_data)

    # Convert scores to binary predictions (good or bad)
    predicted = (scores > 0.8).float().squeeze()

    # Convert labels to float for comparison (if they are not already)
    batch_labels = batch_labels.float().squeeze()

    # Count the number of correct predictions
    correct += (predicted == batch_labels).sum().item()
    total += batch_labels.size(0)

# DEBUG: Print values of correct and total
print("Correct predictions:", correct)
print("Total predictions:", total)

# Compute accuracy
accuracy = correct / total

print(f"The classification accuracy on the test set is: {accuracy*100:.2f}%")


Correct predictions: 1509
Total predictions: 1998
The classification accuracy on the test set is: 75.53%


# **Task** **2**

In [23]:
good_music.shape

(10008, 151)

In [24]:
from torch.utils.data import Dataset, DataLoader

good_music_tensor = torch.tensor(good_music, dtype=torch.long)
# Create Dataset and DataLoader
data_loader = DataLoader(good_music_tensor, batch_size=64, shuffle=True)

# Iterate over the DataLoader to get a single batch
for batch in data_loader:
    # If 'good_music_tensor' includes both features and labels, unpack them
    # If it is just a tensor of features, you will not need to unpack into x, y
    x= batch  # If 'good_music_tensor' is a tensor, this line should be just 'x = batch'
    print(x.shape)  # This will print the shape of the features # This will print the shape of the labels, if they exist
    break  # After printing the shape of the first batch, exit the loop


torch.Size([64, 151])


In [80]:
import torch.nn as nn
import torch
import torch.nn.functional as F

def one_hot_encode(note, num_notes):
  tensor = torch.zeros(num_notes, dtype=torch.float)
  tensor[note] = 1.0
  return tensor
class ComposerBase(nn.Module):

    pass

class Composer(nn.Module):
    def __init__(self, input_dim, lstm_units, num_layers=2):
        super(Composer, self).__init__()
        self.input_dim = input_dim
        self.lstm_units = lstm_units
        self.num_layers = num_layers  # Define the num_layers attribute here
        self.lstm = nn.LSTM(input_dim, lstm_units, num_layers, batch_first=True)
        self.fc = nn.Linear(lstm_units, input_dim)

    def forward(self, x, prev_state):


      state_h, state_c = prev_state


      output, state = self.lstm(x, prev_state)
      logits = self.fc(output)
      return logits, state

    def init_state(self, batch_size):
        # Initialize the hidden and cell state to zeros
        return (torch.zeros(self.num_layers, batch_size, self.lstm_units),
                torch.zeros(self.num_layers, batch_size, self.lstm_units))

    def compose(self, start_sequence, length, temperature=1.0):
        # Ensure that start_sequence is a torch tensor
        if not isinstance(start_sequence, torch.Tensor):
            start_sequence = torch.tensor(start_sequence, dtype=torch.long)

        # Initialize the hidden state
        state_h, state_c = self.init_state(1)  # Batch size of 1 for generation

        # Initialize the sequence with the start_sequence
        current_input = one_hot_encode(start_sequence[0], self.input_dim).unsqueeze(0).unsqueeze(0)

        generated_sequence = start_sequence.tolist()
        # Generate the sequence
        for _ in range(length):
            # Forward pass through LSTM
            output, (state_h, state_c) = self.forward(current_input, (state_h, state_c))

            # Get the last output (next note prediction)
            last_output = output[:, -1, :]

            # Apply temperature scaling and softmax to generate probabilities
            probabilities = F.softmax(last_output / temperature, dim=1).squeeze()

            # Sample from the probability distribution to get the next note
            next_note = torch.multinomial(probabilities, 1).item()

            # Append the predicted note to the sequence
            generated_sequence.append(next_note)
            next_input = one_hot_encode(next_note, self.input_dim).unsqueeze(0).unsqueeze(0)
            current_input = torch.cat((current_input[:, 1:], next_input), 1)

        return generated_sequence[1:]


In [70]:
# Hyperparameters
# Hyperparameters
batch_size = 64
num_epochs = 5
input_dim = good_music.max() + 1  # Assuming good_music is a numpy array, add 1 for zero-indexing

# Convert good_music to a tensor and create the dataset
good_music_tensor = torch.from_numpy(good_music).long()
sequences = good_music_tensor[:, :-1]  # all but the last element for each sequence
targets = good_music_tensor[:, 1:]  # all but the first element for each sequence

dataset = TensorDataset(sequences, targets)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define the Composer model
composer = Composer(input_dim=good_music_tensor.max() + 1, lstm_units=512, num_layers=2)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(composer.parameters(), lr=0.001)



In [71]:
# Training loop
for epoch in range(num_epochs):
    state_h, state_c = composer.init_state(batch_size)
    for batch, (x, y) in enumerate(data_loader):
        optimizer.zero_grad()
        current_batch_size = x.size(0)
        # Initialize the hidden state if it's None or if the batch size has changed (last batch case)
        if state_h is None or state_h.size(1) != current_batch_size:
            state_h, state_c = composer.init_state(current_batch_size)
        x_one_hot = nn.functional.one_hot(x, num_classes=composer.input_dim).float()
        y_pred, (state_h, state_c) = composer(x_one_hot, (state_h, state_c))

        # Flatten the output and targets
        loss = criterion(y_pred.transpose(1, 2), y)

        state_h = state_h.detach()
        state_c = state_c.detach()

        loss.backward()
        optimizer.step()

        print(f'Epoch {epoch}, Batch {batch}, Loss: {loss.item()}')



x shape: torch.Size([64, 150, 382])
state_h shape: torch.Size([2, 64, 512])
state_c shape: torch.Size([2, 64, 512])
Epoch 0, Batch 0, Loss: 5.955317974090576
x shape: torch.Size([64, 150, 382])
state_h shape: torch.Size([2, 64, 512])
state_c shape: torch.Size([2, 64, 512])
Epoch 0, Batch 1, Loss: 5.917027473449707
x shape: torch.Size([64, 150, 382])
state_h shape: torch.Size([2, 64, 512])
state_c shape: torch.Size([2, 64, 512])
Epoch 0, Batch 2, Loss: 5.865227222442627
x shape: torch.Size([64, 150, 382])
state_h shape: torch.Size([2, 64, 512])
state_c shape: torch.Size([2, 64, 512])
Epoch 0, Batch 3, Loss: 5.74569034576416
x shape: torch.Size([64, 150, 382])
state_h shape: torch.Size([2, 64, 512])
state_c shape: torch.Size([2, 64, 512])
Epoch 0, Batch 4, Loss: 5.132244110107422
x shape: torch.Size([64, 150, 382])
state_h shape: torch.Size([2, 64, 512])
state_c shape: torch.Size([2, 64, 512])
Epoch 0, Batch 5, Loss: 4.744360446929932
x shape: torch.Size([64, 150, 382])
state_h shape: to

#Task3

In [82]:
import torch
import random

# 初始化 Composer 实例
composer = Composer(input_dim=good_music_tensor.max() + 1, lstm_units=512, num_layers=2)
composer.eval()  # 设置为评估模式

# 初始化 Critic 实例
model1 = Critic()
model1.eval()  # 设置为评估模式

# 生成序列并评分
generated_sequences = []
scores = []
total_score = 0
for _ in range(50):
    # 随机选择起始音符
    start_note = random.randint(1, good_music_tensor.max())  # 这里假设 good_music_tensor.max() 是合理的起始音符上限
    start_sequence = [start_note]
    start_sequence_tensor = torch.tensor(start_sequence, dtype=torch.long)

    # 使用 Composer 生成序列
    generated_sequence = composer.compose(start_sequence_tensor, length=151)

    # 将生成的序列转换为 Critic 模型所需的张量格式
    generated_sequence_tensor = torch.tensor([generated_sequence], dtype=torch.float)

    # 使用 Critic 模型评分
    score = model1.score(generated_sequence_tensor)
    print(score)
    total_score += score

    # 存储序列及其得分
    generated_sequences.append(generated_sequence)
    scores.append(score)

average_score = total_score / 50
print(f"Average Score: {average_score}")

[[0.5137102]]
[[0.5144695]]
[[0.51318765]]
[[0.5134285]]
[[0.51450986]]
[[0.5143665]]
[[0.51556844]]
[[0.5144438]]
[[0.5152472]]
[[0.51366645]]
[[0.5147051]]
[[0.5164568]]
[[0.5157888]]
[[0.5132117]]
[[0.51514137]]
[[0.5146108]]
[[0.5132778]]
[[0.5129654]]
[[0.5141907]]
[[0.5136273]]
[[0.51288044]]
[[0.5126859]]
[[0.51568747]]
[[0.5146623]]
[[0.5132475]]
[[0.5168321]]
[[0.51456296]]
[[0.5148454]]
[[0.5142295]]
[[0.51358]]
[[0.5133826]]
[[0.513445]]
[[0.51368535]]
[[0.5130595]]
[[0.51421005]]
[[0.51292425]]
[[0.51269853]]
[[0.51516724]]
[[0.51395744]]
[[0.51497036]]
[[0.5140006]]
[[0.51473486]]
[[0.51562166]]
[[0.514067]]
[[0.5160555]]
[[0.5134913]]
[[0.51501155]]
[[0.51512057]]
[[0.5138838]]
[[0.5149354]]
Average Score: [[0.5142842]]
